# TP1 : Linear regression

The purpose of this work is to implement least square linear regression to medical data. The problem is based on an example described in the book by Hastie & Tibshirani (2009) pp. 3-4 & 49-63. Data come from a study published by Stamey et al. (1989). This study aims at the prediction of the level of prostate specific antigen, denoted by `lpsa` below, from the
results of clinical exams. These exams are carried out before a possible
prostatectomy.

The measurements are log cancer volume `lcavol`, log prostate 
weight `lweight`, age of the patient `age`, log of benign prostatic 
hyperplasia amount `lbph`, seminal vesicle invasion `svi`, log of capsular 
penetration `lcp`, Gleason score `gleason`, and percent of Gleason scores 4 or 
5 `pgg45`. The variables `svi` and `gleason` are categorical, others are
quantitative. There are `p=8` entries.
The work is decomposed in the following tasks:

* read and format the data : extraction of the training and test sets,
* apply least square regression method to predict `lpsa` from the entries,
* study the estimated error on the test set (validation),
* identify the most significant entries by using a rejection test,
* apply regularized least square regression method (ridge regression),
* search for an optimal regularization parameter thanks to
cross-validation.

In [1]:
import csv
# import os
from pylab import *
import numpy as np
from numpy import linalg as la

import pandas as pd
import seaborn as sns

## Read & Normalize data
Data are stored in ASCII format: 

* the first column enumerates the data from 1 à 97 (97 male subjects). 
* columns 2 to 9 contain the entries themselves. 
* column 10 contains target values. 
* column 11 contains label 1 for the training set, 
and 2 for the test set.

In [2]:
#%% To read data from spaced separated float numbers
# x, y = np.loadtxt(c, delimiter=',', usecols=(0, 2), unpack=True)

data_init = np.loadtxt('prostate_data_sansheader.txt')

data = data_init[:,1:]   # we get rid of the indices (1 to 97)

#%% Extraction of training/test sets
Itrain = np.nonzero(data[:,-1]==1)
data_train=data[Itrain]   # original data

Itest = np.nonzero(data[:,-1]==0)
data_test = data[Itest]   # original data

**Normalization of the data** *with respect to the mean and standard deviation of the training set*.

In [3]:
M_train = data_train
M_test = data_test 
moy = np.zeros((8,))
sigma = np.zeros((8,))

# With a FOR loop :
for k in range(8): # 8 columns of entries
    moy[k]=np.mean(data_train[:,k])
    sigma[k] = np.std(data_train[:,k], ddof=0)
    M_train[:,k] = (data_train[:,k]-moy[k])/sigma[k] # normalized: centered, variance 1
    M_test[:,k] = (data_test[:,k]-moy[k])/sigma[k]   # same normalization for test set

print(sigma)

[ 1.23328245  0.47303067  7.44601122  1.45269103  0.41684299  1.39024269
  0.70355366 29.08227243]


In [24]:
# Alternative WITHOUT FOR
normalize = lambda vec: (vec-np.mean(vec))/np.std(vec)    # inline function 
M_train = np.array( [ normalize(vec) for vec in data_train[:,0:8].T ] ).T  # iterate on vec direct / ARRAY not LIST
moy = np.array( [ np.mean(vec) for vec in data_train[:,0:8].T ] )
sigma = np.array( [ np.std(vec, ddof=0) for vec in data_train[:,0:8].T ] )

M_test = np.array([ (data_test[:,k]-moy[k])/sigma[k] for k in range(M_train.shape[1]) ] ).T

# Part 1 : simple least square regression

## Preliminary questions
 
 * Compute the autocovariance matrix from the training set.
 * Observe carefully & Comment. What kind of information can you get ? 

In [133]:
# Covariance_matrix=np.cov(np.concatenate((np.array([t_train]),M_train)))
# sns.heatmap(Covariance_matrix)
M_train


array([[-1.53517959, -1.81097918, -1.98042525, -1.00347165, -0.53708616,
        -0.84308399, -1.03949894, -0.90325324, -0.4307829 ,  1.        ],
       [-1.87122084, -0.64791082, -0.90602451, -1.00347165, -0.53708616,
        -0.84308399, -1.03949894, -0.90325324, -0.1625189 ,  1.        ],
       [-1.47923712, -1.97632998,  1.24277698, -1.00347165, -0.53708616,
        -0.84308399,  0.38185676, -0.21554907, -0.1625189 ,  1.        ],
       [-2.04127153, -0.72578526, -0.90602451, -1.00347165, -0.53708616,
        -0.84308399, -1.03949894, -0.90325324, -0.1625189 ,  1.        ],
       [-0.45575567, -0.40956052, -0.36882414, -1.00347165, -0.53708616,
        -0.84308399, -1.03949894, -0.90325324,  0.3715636 ,  1.        ],
       [-1.91627934, -0.83986455, -1.98042525, -1.00347165, -0.53708616,
        -0.84308399, -1.03949894, -0.90325324,  0.7654678 ,  1.        ],
       [-0.50300267, -0.18307203, -0.90602451,  1.00876737, -0.53708616,
        -0.84308399, -1.03949894, -0.90325324

On observe que la majorité de données sont décorrélées car les coefficients non diagonaux sont faibles. 

Quelques rappels à propos de la notion de covariance : https://fr.wikipedia.org/wiki/Covariance

## Exercise 1 : least square regression 
 * Build the matrix of features `X_train` for the training set, the first column is made of ones.
 * Estimate the regression vector `beta_hat` (estimates= `X*beta_hat`)
 _Indication: you may either use the function `inv` or another more efficient way to compute $A^{-1}B$ (think of `A\B`)._ 
 * What is the value of the first coefficient `beta_hat[0]` ? What does it correspond to ?
 * Estimate the prediction error (quadratic error) from the test set.


*Indication: be careful of using `X_test` defined above, normalized w.r.t. the training data set. You can estimate this error by using:*

In [4]:
t_test = data_test[:,8]   # target column
N_test = data_test.shape[0]
X_test = np.concatenate((np.ones((N_test,1)), M_test[:,0:8]), axis=1) 
# don't forget the 1st column of ones and normalization !

In [5]:
# Exercise 1
#Build the matrix of features X_train
t_train = data_train[:,8]   # target column
N_train = data_train.shape[0]
X_train = np.concatenate((np.ones((N_train,1)), M_train[:,0:8]), axis=1) 
#On construit X_train en rajoutant la colonne des 1 pour avoir X_train tild et en normalisant le vecteur 

print (f"Le rang de la X_train est : {np.linalg.matrix_rank(X_train)}")

#X_train est de rang plein donc on pourra calculer sa pseudo inverse de Moore Penrose dans la suite.

Le rang de la X_train est : 9


$MP = (X^{T}X)^{-1}X$

In [6]:
MP = np.dot(np.linalg.inv(np.dot(X_train.T,X_train)),X_train.T) #Pseudo inverse de Moore Penrose
 
beta_hat=MP@t_train
print(f" Le premier coefficient b0 est le biais du model et il vaut : {np.round(beta_hat[0],3)}")

 Le premier coefficient b0 est le biais du model et il vaut : 2.452


In [7]:
#La prédiction de l'erreur:

t_test_hat = X_test@beta_hat
Error_test = t_test_hat-t_test

print(f"L'erreur quadratique du modèle est de {np.round(np.linalg.norm(Error_test,2)**2,3)}")

L'erreur quadratique du modèle est de 15.638


## Rejection test, computation of Z-scores
Now we turn to the selection of the most significant entries so that our predictor be more robust. The essential idea is that our estimates will be more robust if only the most significant entries are taken into account. As a consequence, note that we will *reduce the dimension* of the problem from |p=8| to some smaller dimension. The present approach uses a statistical test to decide whether the regression coefficient corresponding to some entry is significantly non-zero. Then we can decide either to put non significant coefficients to zero, or to select the significant entries only and estimate the new reduced regression vector.

Let's assume that target values are noisy due to some white Gaussian
noise with variance $\sigma^2$ (see Hastie & Tibshirani p. 47). One can show that the estimated regression vector |beta_hat| is also Gaussian with variance

$$ var (\widehat{\beta}) = (X^TX)^{-1}\sigma^2.$$  

One can also show that the estimator of the variance (from the training set)

$$\widehat{\sigma^2}=\frac{1}{(N-p-1)}\sum (t_n-\widehat{t}_n)^2$$

obeys a Chi-2 distribution. As a consequence a Chi-square statistical test can be used to determine whether some coefficient $\beta_j$ is
significantly non-zero. To this aim, one defines the variables $z_j$
named Z-scores which in turn obey a Fisher law, also called
$t$-distribution, which are often used in statistics:

$$ z_j = \frac{\beta_j}{\widehat{\sigma}\sqrt{v_j}} $$

where $v_j$ is the $j$-th diagonal element of the matrix $(X^TX)^{-1}$.
For sake of simplicity, we will consider that the null hypothesis of
$\beta_j$ is rejected with probability 95% if the Z-score is greater than 2.

## Exercise 2
1. Compute the Z-scores and select the most significant entries.
2. Estimate the prediction error over the test set if only these significant 
entries are taken into account for regression by putting other regression 
coefficients to zero.
3. Estimate the new regression vector when only the significant features
are taken into account.
4. Compare to previous results (Exercise 1).

*Indication 1 : to sort a vector `Z` in descending order*
`val = np.sort(np.abs(Z))[-1:0:-1]`


*Indication 2 :* to extract the diagonal of a matrix,
`vXX = np.diag(inv(X.T.dot(X),k=0)`

In [8]:
# Exercise 2
import pdb
def var_GN(X):
    return(np.diag(inv(X.T.dot(X))))

def sigma_hat(beta_hat,X,t):
    p=len(beta_hat)-1 #On enlève 1 pour enlever le biais
    N=len(X)
    t_hat=X@beta_hat
           
    return(np.sqrt(np.linalg.norm((t-t_hat),2)**2/(N-p-1)))
    
def z_score(X,beta_hat,t):
    Z=[]
    Var = var_GN(X)
    s_hat=sigma_hat(beta_hat,X,t)
    for j in range(0,len(beta_hat)):
        Z.append(beta_hat[j]/(s_hat*np.sqrt(Var[j])))
    return(Z)

Z_score=z_score(X_train,beta_hat,t_train)

df=pd.DataFrame(np.abs(Z_score),columns=["Z_score"])
df=df.sort_values(by=["Z_score"])
Index_selected = np.array(df[df["Z_score"]<=2].index)
Index_selected

array([7, 3, 8, 6], dtype=int64)

In [19]:
t_train_selected = data_train[:,8]  # target column
N_train_selected = t_train_selected.shape[0]
X_train_selected = np.concatenate((np.ones((N_train_selected,1)), M_train[:,Index_selected]), axis=1)

print (f"Le rang de la X_train_selected est : {np.linalg.matrix_rank(X_train_selected)}")


Le rang de la X_train_selected est : 5


In [20]:
np.shape(t_train_selected)

(67,)

In [23]:
MP_selected = np.dot(np.linalg.inv(np.dot(X_train_selected.T,X_train_selected)),X_train_selected.T) #Pseudo inverse de Moore Penrose
 
beta_hat_selected=MP_selected@t_train_selected
print(f" Le premier coefficient b0 est le biais du model et il vaut : {beta_hat_selected[0]}")

 Le premier coefficient b0 est le biais du model et il vaut : -7.216449660063518e-15


In [25]:
beta_hat_selected

array([-7.21644966e-15, -8.32667268e-17, -1.66533454e-16,  1.00000000e+00,
        1.66533454e-16])

# Part 2: Regularized least squares
This part deals with regularized least square regression. We denote
by `beta_hat_reg` the resulting coefficients. This approach is an alternative to the selection based on statistical tests above. The idea is now to penalize large values of regression coefficients, *except for the bias*.

We use the result:

$$\hat{\beta} = (\lambda I_p + X_c^T X_c)^{-1} X_c^T t_c$$

where $X_c$ contains the normalized entries of the training data set with 
no column of ones (the bias should no be penalized and is processed). 
The targets `t_c` are therefore also centered, `t_c=t-mean(t)`.
 
First, we estimate the bias $t_0$ to center the targets which yields the coefficient $\beta_0$, that is `beta_hat_reg[0]` in Python.

*Remark : the bias is estimated as the empirical average of targets.
For tests, entries should be normalized with respect to the means and
variances of the training data set (see exercise 3.5 p. 95 in Hastie & Tibshirani). Then work on the vector of entries with no column of ones.*

## Exercise 3
1. Use _ridge regression_ for penalty `lambda = 25` to estimate the regression vector. 
2. Estimate the prediction error from the test set.
3. Compare the results (coefficients $\beta$, error...) to previous ones.
4. You may also compare these results to the result of best subset selection below:

`beta_best = [2.477 0.74 0.316 0 0 0 0 0 0]`.

*Indication : a simple way to obtain predictions for the test data set is the code below:*

In [27]:
t = data_train[:,8]   # column of targets
t0 = np.mean(t)

N_test = data_test.shape[0]
X_test = np.hstack((np.ones((N_test,1)), M_test[:,0:8]))  
t_test = data_test[:,8]  


In [ ]:
# Here the 1st column of X_test is a column of ones.
t_hat_reg = X_test.dot(beta_hat_reg)

# Part 3: Cross-Validation 

## How to choose lambda from the training data set only ? 

The idea is to decompose the training set in 2 subsets: one subset for
linear regression (say 9/10 of the data), the other to estimate the prediction error (say 1/10 of the data).

We can repeat this operation 10 times over the 10 possible couples of
subsets to estimate the average prediction error. We will choose the
value of `lambda` which minimizes this error. The algorithm goes as
follows:

For the 10 cross-validation cases
    
    Extraction of test & training subsets `testset` & `trainset`
    
    For lambda in 0:40
        Estimate `beta_hat` from normalized `trainset` (mean=0, var=1)
        Estimate the error from  `testset`
    EndFor lambda

EndFor 10 cases

Compute the average error for each lambda

Choose `lambda` which minimizes the error 


## Exercise 4
* Use 6-fold cross-validation in the present study to optimize the choice of `lambda`. 
Try values of `lambda` ranging from 0 to 40 for instance (0:40).
* Plot the estimated error as a function of `lambda`.
* Propose a well chosen value of `lambda` and give the estimated corresponding
error on the test set.
* Comment on your results.

*Indication 1 : think of shuffling the dataset first.*

*Indication 2 : you can build 6 training and test subsets by using the code below*

In [ ]:
lmax = 40
lambda_pos = arange(0,lmax+1) 

N_test = 10
m=np.zeros(8)
s = np.zeros(8)
X_traink = np.zeros((X_train.shape[0]-N_test,8))
X_testk = np.zeros((N_test,8))
erreur = np.zeros((6,lmax+1))
erreur_rel = np.zeros((6,lmax+1))

In [ ]:
# Exercise 4 
for p in range(6):   # loop on test subsets
    # extraction of testset
    testset  = data_train[arange(p*N_test,(p+1)*N_test),0:9] 
    # extraction of trainset
    trainset = data_train[hstack((arange(p*N_test),arange((p+1)*N_test,data_train.shape[0]))),0:9] 
    # normalization of entries, 
    # ...

In [ ]:
# Exercise 4 ---------------
# ...
# averaged error on the 6 training/test sets ?
# averaged error on the 6 training/test sets ?
# standard variation of this error estimate ?

# print(erreur_lambda, std_erreur_lambda, erreur_rel_lambda)


In [ ]:
# Exercise 4 FIGURE ---------------
# ...


In [ ]:
# Exercise 4 (continued)